<a href="https://colab.research.google.com/github/ccnmagnoo/colabs/blob/main/sec_etl_professional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Resume
Actually there´s 11.700 profesional certificated in GAS and Electricity procedures. But Just 290 of them are women, and [SEC](www.sec.cl) (institution focused in regulate this profesionals), dosen´t focus in this labor. This code is used for extract and filter those women to use in diverse social programs, and visualice their labor and effort in this energy field

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
SOURCE_WEB = 'https://wlhttp.sec.cl/buscadorinstaladores/buscador.do'
PATH_GDRIVE = '/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality'
eli_source = f'{PATH_GDRIVE}/sec_eli.xls'
gas_source = f'{PATH_GDRIVE}/sec_gas.xls'

#Preparing Dataset

In [ ]:
#read gas & dataframe it
gas_df = pd.read_excel(gas_source,header=4)
gas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1898 entries, 0 to 1897
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NOMBRE              1898 non-null   object
 1   CORREO ELECTRÓNICO  1867 non-null   object
 2   RUT                 1898 non-null   object
 3   DIRECCIÓN           1898 non-null   object
 4   TELÉFONO            902 non-null    object
 5   CELULAR             1844 non-null   object
 6   LICENCIA CLASE      1898 non-null   object
dtypes: object(7)
memory usage: 103.9+ KB


In [ ]:
#read electrica & dataframe it
eli_df = pd.read_excel(eli_source,header=4)
eli_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9881 entries, 0 to 9880
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NOMBRE              9881 non-null   object
 1   CORREO ELECTRÓNICO  9611 non-null   object
 2   RUT                 9881 non-null   object
 3   DIRECCIÓN           9881 non-null   object
 4   TELÉFONO            4465 non-null   object
 5   CELULAR             9614 non-null   object
 6   LICENCIA CLASE      9881 non-null   object
dtypes: object(7)
memory usage: 540.5+ KB


In [ ]:
#joint gas + eli
all_technicians = pd.concat([eli_df,gas_df])
all_technicians.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11779 entries, 0 to 1897
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NOMBRE              11779 non-null  object
 1   CORREO ELECTRÓNICO  11478 non-null  object
 2   RUT                 11779 non-null  object
 3   DIRECCIÓN           11779 non-null  object
 4   TELÉFONO            5367 non-null   object
 5   CELULAR             11458 non-null  object
 6   LICENCIA CLASE      11779 non-null  object
dtypes: object(7)
memory usage: 736.2+ KB


#Definig Transform Functions

In [ ]:
from pandas.io.formats.style_render import non_reducing_slice
from pandas.core.arrays import boolean
#name normalizer
from typing import Union, List
from enum import Enum
import numpy as np


#IMPORTANT : constant used in keras model version 2
CHARS:List[str] = list('aábcdeéfghiíjklmnñoópqrstuúvwxyz')
NAME_BLOCK_SIZE = 20

class Gender(Enum):
  MALE = 'male'
  FEMALE='female'

#normalize name for keras neuronal net consumption
def nameSerializer(unitName:str,size:int=20)->Union[List[float],None]:
  if unitName == None:
    return None

  name_array = list(unitName)
  # [f,i,r,s,t] ->[6/23,8/23,9/23,11/23,20/23] indexation
  descriteName = list(map(lambda char:CHARS.index(char)/len(CHARS),name_array))
  # content into a 20-size box [0.0 ,0.0 ,....0.0]
  box = [0.0]*(size-len(descriteName))
  input = np.concatenate((np.array(box),np.array(descriteName)))
  return list(input.flatten())

#extract just the first name without divisions
class Composer(Enum):
  FIRST = 0
  SECOND = 1

def nameNormalizer(input:str|None,compose:Composer=Composer.FIRST)->Union[str,None]:
  #single word name
  if input == None:
    return None
  #single name checker
  #if len(input.split(' '))==1 and compose == Composer.SECOND:
    #compose = Composer.FIRST

  decomposed_name:str = input.strip().split(' ')[compose.value if len(input.split(' '))>1 else 0].lower()
  #remove al not alpha
  return ''.join([ char for char in decomposed_name if char.isalpha()])

#import ML keras tensorflow model
import keras
from keras.optimizers import get
MODEL = 'keras_genderizer_model_v2.h5'
MODEL_PATH = f'/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality/{MODEL}'
model = keras.models.load_model(MODEL_PATH)

#wrapping gender
def getGenderByKeras(unitName:str|None)->Union[Gender,None]:
  if unitName == None:
    return None

  options=[Gender.MALE,Gender.FEMALE]
  input = nameSerializer(unitName,20)

  prediction = model.predict([input])[0]
  category = options[np.argmax(prediction)]
  return category

#for saving process simplified selector with just "o" or "a"
def getGenderSimplified(unitName:str|None)->Gender | None:
  if not unitName:
    return None

  last_char = unitName[-1]
  if last_char == 'o':
    return Gender.MALE
  if last_char == 'a':
    return Gender.FEMALE
  return None

from pandas import DataFrame
WRONG_POSITIVE:DataFrame = pd.read_excel('/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality/wrong_positive.xlsx')

def check_is_male_name(unitName:str|None)-> bool:
  if not unitName:
    return False

  return WRONG_POSITIVE['nombre'].str.contains(unitName).any()

#main general function wrapper⭐
def getGender(name:str | None)->Gender| str | None:
  #checking input is empty
  if name ==  None:
    return None

  #catching wrong positive
  if check_is_male_name(nameNormalizer(name)):
    return Gender.MALE.value

  #try simplified calculus, for speed 🚀
  firstname_check = getGenderSimplified(nameNormalizer(name))
  if firstname_check:
    return firstname_check.value

  #second try with second name
  second = getGenderSimplified(nameNormalizer(name,Composer.SECOND))
  if second:
    return second.value


  #if simplified dosen´t work applys de keras model
  modeler  = getGenderByKeras(nameNormalizer(name))
  if modeler:
    return modeler.value

  return None

In [ ]:
len('Carlos'.split(' '))
getGender('Joshua')

'male'

#Transform Dataset & calculating columns

In [ ]:
# Transform
## split NOMBRE column
all_technicians[['Apellidos','Nombres']] = all_technicians['NOMBRE'].str.split(',',1,expand=True)
## remove null names
all_technicians['Nombres'] = all_technicians['Nombres'].map(lambda name: name.strip())
all_technicians['DIRECCIÓN'] = all_technicians['DIRECCIÓN'].str.replace('Nunoa','Ñuñoa')
all_technicians

<ipython-input-70-939b2ce165c1>:3: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  all_technicians[['Apellidos','Nombres']] = all_technicians['NOMBRE'].str.split(',',1,expand=True)


,NOMBRE,CORREO ELECTRÓNICO,RUT,DIRECCIÓN,TELÉFONO,CELULAR,LICENCIA CLASE,Apellidos,Nombres
0,"Moraga Núñez , Jorge Orlando",jmoragan@gmail.com,7324296-7,"Cerro Altar 6616, 601, Las Condes",25774233,73381253,A,Moraga Núñez,Jorge Orlando
1,"Sánchez Jorquera , Carlos Mauricio",carlos.msanchezj@gmail.com,18328350-2,"colina 4915, Conchalí",NaN,56940447001,A,Sánchez Jorquera,Carlos Mauricio
2,"San Martín Salamanca , Felipe Andrés",felipe.sanmartin.salamanca@gmail.com,18101438-5,"Eleuterio Ramirez 1241, Pobl. 2 de Septiembre,...",NaN,934113388,B,San Martín Salamanca,Felipe Andrés
3,"Calderón Barraza , Nabor Alejandro",ncalderon@cjenergy.cl,13977385-3,"avenida rica aventura 11544, Antofagasta",055950899,099591296,A,Calderón Barraza,Nabor Alejandro
4,"Araya Reyes , Jecar Exequiel",jecar.araya@gmail.com,17478701-8,"Pasaje 1 62, Nueva Aurora, Viña del Mar, Vina ...",NaN,956841025,B,Araya Reyes,Jecar Exequiel
...,...,...,...,...,...,...,...,...,...
1893,"Hernández Araya , Alexis Duvan",alexishernandez.araya@gmail.com,19612379-2,"Avenida los puelches 921, Chillan",NaN,978706209,1,Hernández Araya,Alexis Duvan
1894,"Rivas Navarrete , Miguel Eduardo",alambreart@gmail.com,17747817-2,"mulchen 383, instalador de redes de gas ,clase...",413351174,930893107,3,Rivas Navarrete,Miguel Eduardo
1895,"Quinillao Carvallo , Henrri Aníbal",henrriq@gmail.com,13734890-K,"malvoa 939, Temuco",NaN,97155737,3,Quinillao Carvallo,Henrri Aníbal
1896,"Sunkel Ojeda , Juan Carlos",jcsunkel@gmail.com,7641852-7,"Pasaje Navidad 114, Villa Cataluña, Los Ángeles",NaN,978521408,1,Sunkel Ojeda,Juan Carlos


##Detecting name gender

In [ ]:
all_technicians['gender'] = all_technicians['Nombres'].map(getGender)

In [ ]:
#big filter
from pandas import DataFrame
#result:DataFrame = all_technicians.loc[all_technicians['gender']=='female']
result:DataFrame = all_technicians
result

,NOMBRE,CORREO ELECTRÓNICO,RUT,DIRECCIÓN,TELÉFONO,CELULAR,LICENCIA CLASE,Apellidos,Nombres,gender
0,"Moraga Núñez , Jorge Orlando",jmoragan@gmail.com,7324296-7,"Cerro Altar 6616, 601, Las Condes",25774233,73381253,A,Moraga Núñez,Jorge Orlando,male
1,"Sánchez Jorquera , Carlos Mauricio",carlos.msanchezj@gmail.com,18328350-2,"colina 4915, Conchalí",NaN,56940447001,A,Sánchez Jorquera,Carlos Mauricio,male
2,"San Martín Salamanca , Felipe Andrés",felipe.sanmartin.salamanca@gmail.com,18101438-5,"Eleuterio Ramirez 1241, Pobl. 2 de Septiembre,...",NaN,934113388,B,San Martín Salamanca,Felipe Andrés,male
3,"Calderón Barraza , Nabor Alejandro",ncalderon@cjenergy.cl,13977385-3,"avenida rica aventura 11544, Antofagasta",055950899,099591296,A,Calderón Barraza,Nabor Alejandro,male
4,"Araya Reyes , Jecar Exequiel",jecar.araya@gmail.com,17478701-8,"Pasaje 1 62, Nueva Aurora, Viña del Mar, Vina ...",NaN,956841025,B,Araya Reyes,Jecar Exequiel,male
...,...,...,...,...,...,...,...,...,...,...
1893,"Hernández Araya , Alexis Duvan",alexishernandez.araya@gmail.com,19612379-2,"Avenida los puelches 921, Chillan",NaN,978706209,1,Hernández Araya,Alexis Duvan,male
1894,"Rivas Navarrete , Miguel Eduardo",alambreart@gmail.com,17747817-2,"mulchen 383, instalador de redes de gas ,clase...",413351174,930893107,3,Rivas Navarrete,Miguel Eduardo,male
1895,"Quinillao Carvallo , Henrri Aníbal",henrriq@gmail.com,13734890-K,"malvoa 939, Temuco",NaN,97155737,3,Quinillao Carvallo,Henrri Aníbal,male
1896,"Sunkel Ojeda , Juan Carlos",jcsunkel@gmail.com,7641852-7,"Pasaje Navidad 114, Villa Cataluña, Los Ángeles",NaN,978521408,1,Sunkel Ojeda,Juan Carlos,male


In [ ]:
#transform functions

#extract city
def extract_last_col(dir:str)->str|None:
  dir_list = list(reversed(dir.split(',')))
  if len(dir_list)==0:
    return None
  return dir_list[0]


#name case format
import re

def namecase(string:str)->str:
  s_list = re.split('\s+',string.strip())
  for index, word in enumerate(s_list):
    s_list[index] = word[0].upper() + word[1:].lower()
  return ' '.join(s_list)

def setType(input_type:str)->str:
  if input_type == '1 y 4':
    return 'gas'
  if input_type.isnumeric():
    return 'gas'
  else:
    return 'eli'

def getAge(rut:str)->int:
  PENDANT = 3.3363697569700348e-06
  INTERCEPT = 1932.2573852507373
  rut_compose:list[str] = rut.split('-')
  return int(int(rut_compose[0])*PENDANT+INTERCEPT)


  return 0

#Get Territory
from difflib import SequenceMatcher
DPA_PATH = '/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality/CUT2018.xls'
DPA = pd.read_excel(DPA_PATH)
from pandas import Series

class Territory(Enum):
  CITY='city'
  PROVINCE='province'
  REGION='region'

def getTerritory (query:str,req:Territory=Territory.CITY,res:Territory=Territory.REGION)->Series:
  #generate a serie with ratio similatities value
  ratio_series:DataFrame = DPA[req.value].apply(lambda req: SequenceMatcher(None,query,req).ratio())
  #find the most simliar ratio an get this index
  most_similar_index = ratio_series.idxmax()
  #extract row index
  result:Series = DPA.iloc[most_similar_index]
  #return value require, like city, region or province
  return result[res.value]

import math
def normalizePhone(input:int|None)->int|None:

  if len(str(input)) == len('56948611376'):
    return input
  if len(str(input)) == len('948611376'):
    return int('56'+str(input))
  if len(str(input)) == len('48611376'):
    return int('569'+str(input))
  return None



In [ ]:
getTerritory('Quillomta',res=Territory.REGION)

'Valparaíso'

In [ ]:

result['movil'] = result['CELULAR']
result['movil'] = result['movil'].str.replace(' ','')
result['movil'] = result['movil'].str.replace('-','')
result['movil'] = pd.to_numeric(result['movil'], errors='coerce').fillna(0).astype(int)
result['movil'] = result['movil'].apply(lambda c:normalizePhone(c))

result['phone'] = result['TELÉFONO']
result['phone'] = result['phone'].str.replace(' ','')
result['phone'] = result['phone'].str.replace('-','')
result['phone'] = pd.to_numeric(result['phone'], errors='coerce').fillna(0).astype(int)
result['phone'] = result['phone'].apply(lambda c:normalizePhone(c))

result.head(3)


,NOMBRE,CORREO ELECTRÓNICO,RUT,DIRECCIÓN,TELÉFONO,CELULAR,LICENCIA CLASE,Apellidos,Nombres,gender,Movil,Comuna,Región,Nac Aprox,Categoría
0,"Moraga Núñez , Jorge Orlando",jmoragan@gmail.com,7324296-7,"Cerro Altar 6616, 601, Las Condes",25774233,73381253,A,Moraga Núñez,Jorge Orlando,male,5.697338e+10,Las Condes,Metropolitana de Santiago,1956,eli
1,"Sánchez Jorquera , Carlos Mauricio",carlos.msanchezj@gmail.com,18328350-2,"Colina 4915, Conchalí",NaN,56940447001,A,Sánchez Jorquera,Carlos Mauricio,male,5.694045e+10,Conchalí,Metropolitana de Santiago,1993,eli
2,"San Martín Salamanca , Felipe Andrés",felipe.sanmartin.salamanca@gmail.com,18101438-5,"Eleuterio Ramirez 1241, Pobl. 2 De Septiembre,...",NaN,934113388,B,San Martín Salamanca,Felipe Andrés,male,5.693411e+10,Los Angeles,Biobío,1992,eli


In [ ]:
result['city'] = result['DIRECCIÓN'].map(extract_last_col)
result['city'] = result['city'].apply(lambda c:getTerritory(c,res=Territory.CITY))
result['region'] = result['city'].apply(lambda c:getTerritory(c,res=Territory.REGION))
result['birth'] = result['RUT'].apply(getAge)
result['email'] = result['CORREO ELECTRÓNICO'].str.lower()

result['service'] = result['LICENCIA CLASE'].map(setType)
for field in ['Nombres','Apellidos','DIRECCIÓN','Comuna']:
  result[field] = result[field].map(namecase)

result

,NOMBRE,CORREO ELECTRÓNICO,RUT,DIRECCIÓN,TELÉFONO,CELULAR,LICENCIA CLASE,Apellidos,Nombres,gender,Movil,Comuna,Región,Nac Aprox,Categoría
0,"Moraga Núñez , Jorge Orlando",jmoragan@gmail.com,7324296-7,"Cerro Altar 6616, 601, Las Condes",25774233,73381253,A,Moraga Núñez,Jorge Orlando,male,5.697338e+10,Las Condes,Metropolitana de Santiago,1956,eli
1,"Sánchez Jorquera , Carlos Mauricio",carlos.msanchezj@gmail.com,18328350-2,"Colina 4915, Conchalí",NaN,56940447001,A,Sánchez Jorquera,Carlos Mauricio,male,5.694045e+10,Conchalí,Metropolitana de Santiago,1993,eli
2,"San Martín Salamanca , Felipe Andrés",felipe.sanmartin.salamanca@gmail.com,18101438-5,"Eleuterio Ramirez 1241, Pobl. 2 De Septiembre,...",NaN,934113388,B,San Martín Salamanca,Felipe Andrés,male,5.693411e+10,Los Angeles,Biobío,1992,eli
3,"Calderón Barraza , Nabor Alejandro",ncalderon@cjenergy.cl,13977385-3,"Avenida Rica Aventura 11544, Antofagasta",055950899,099591296,A,Calderón Barraza,Nabor Alejandro,male,5.699959e+10,Antofagasta,Antofagasta,1978,eli
4,"Araya Reyes , Jecar Exequiel",jecar.araya@gmail.com,17478701-8,"Pasaje 1 62, Nueva Aurora, Viña Del Mar, Vina ...",NaN,956841025,B,Araya Reyes,Jecar Exequiel,male,5.695684e+10,Viña Del Mar,Valparaíso,1990,eli
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,"Hernández Araya , Alexis Duvan",alexishernandez.araya@gmail.com,19612379-2,"Avenida Los Puelches 921, Chillan",NaN,978706209,1,Hernández Araya,Alexis Duvan,male,5.697871e+10,Chillán,Ñuble,1997,gas
1894,"Rivas Navarrete , Miguel Eduardo",alambreart@gmail.com,17747817-2,"Mulchen 383, Instalador De Redes De Gas ,clase...",413351174,930893107,3,Rivas Navarrete,Miguel Eduardo,male,5.693089e+10,San Pedro De La Paz,Biobío,1991,gas
1895,"Quinillao Carvallo , Henrri Aníbal",henrriq@gmail.com,13734890-K,"Malvoa 939, Temuco",NaN,97155737,3,Quinillao Carvallo,Henrri Aníbal,male,5.699716e+10,Temuco,La Araucanía,1978,gas
1896,"Sunkel Ojeda , Juan Carlos",jcsunkel@gmail.com,7641852-7,"Pasaje Navidad 114, Villa Cataluña, Los Ángeles",NaN,978521408,1,Sunkel Ojeda,Juan Carlos,male,5.697852e+10,Los Angeles,Biobío,1957,gas


In [ ]:
result_path = f'{PATH_GDRIVE}/sec_result.xlsx'
result.to_excel(result_path)

In [ ]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11779 entries, 0 to 1897
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   NOMBRE              11779 non-null  object 
 1   CORREO ELECTRÓNICO  11478 non-null  object 
 2   RUT                 11779 non-null  object 
 3   DIRECCIÓN           11779 non-null  object 
 4   TELÉFONO            5367 non-null   object 
 5   CELULAR             11458 non-null  object 
 6   LICENCIA CLASE      11779 non-null  object 
 7   Apellidos           11779 non-null  object 
 8   Nombres             11779 non-null  object 
 9   gender              11779 non-null  object 
 10  Movil               11038 non-null  float64
 11  Comuna              11779 non-null  object 
 12  Región              11779 non-null  object 
 13  Nac Aprox           11779 non-null  int64  
 14  Categoría           11779 non-null  object 
dtypes: float64(1), int64(1), object(13)
memory usage: 1.4+

Reloading from stored result

In [23]:
reload_path = f'{PATH_GDRIVE}/sec_result.xlsx'
reloaded = pd.read_excel(reload_path)

Filter regional (*optional*)

In [ ]:
result_path_v = f'{PATH_GDRIVE}/sec_result_v.xlsx'
reloaded.loc[reload['region']=='Valparaíso'].to_excel(result_path_v)

,Unnamed: 0,NOMBRE,CORREO ELECTRÓNICO,RUT,DIRECCIÓN,TELÉFONO,CELULAR,LICENCIA CLASE,Apellidos,Nombres,gender,Movil,Comuna,Región,Nac Aprox,Categoría
0,0,"Moraga Núñez , Jorge Orlando",jmoragan@gmail.com,7324296-7,"Cerro Altar 6616, 601, Las Condes",25774233,73381253,A,Moraga Núñez,Jorge Orlando,male,5.697338e+10,Las Condes,Metropolitana de Santiago,1956,eli
1,1,"Sánchez Jorquera , Carlos Mauricio",carlos.msanchezj@gmail.com,18328350-2,"Colina 4915, Conchalí",NaN,56940447001,A,Sánchez Jorquera,Carlos Mauricio,male,5.694045e+10,Conchalí,Metropolitana de Santiago,1993,eli
2,2,"San Martín Salamanca , Felipe Andrés",felipe.sanmartin.salamanca@gmail.com,18101438-5,"Eleuterio Ramirez 1241, Pobl. 2 De Septiembre,...",NaN,934113388,B,San Martín Salamanca,Felipe Andrés,male,5.693411e+10,Los Angeles,Biobío,1992,eli


In [25]:
#removing fat
reloaded = reloaded.drop('NOMBRE',axis =1 )
reloaded = reload.drop('CELULAR',axis =1 )

In [28]:
reload.sample(5)

,Unnamed: 0,email,rut,address,phone,licence,Surname,names,gender,movil,city,region,birth,service
3346,3346,emauricio.rojas@gmail.com,16737596-0,"Las Palmas 65, Villa San Judas Tadeo, Malloa",976053172,B,Rojas Mañán,Edwin Mauricio,male,5.697605e+10,Malloa,Libertador General Bernardo O'Higgins,1988,eli
9418,9418,patriciomolina019@gmail.com,18207270-2,"Ramirez 2297, Villa Abedules, La Union",967615944,A,Molina Reyes,Patricio Antonio,male,5.696762e+10,La Unión,Los Ríos,1993,eli
1971,1971,moi.solar29@gmail.com,18326150-9,"Cerro Nevado 787, Quilicura",NaN,D,Solar Zúñiga,Moisés Alejandro,male,5.693011e+10,Quilicura,Metropolitana de Santiago,1993,eli
9621,9621,bonifaciocuri@gmail.com,16464975-K,"Poeta Juan Luis Martínez 1758, Santa María De ...",NaN,B,Maldonado Maldonado,Denis Marcelo,male,5.699843e+10,Colina,Metropolitana de Santiago,1987,eli
409,409,riego.agric@gmail.com,10791345-9,"Esteban Grlica 632, Villa Las Salinas Sindempa...",051289194,B,Rojas Cisternas,Jorge Wilson,male,5.696850e+10,Coquimbo,Coquimbo,1968,eli


In [29]:
reloaded.to_excel(reload_path)

#Bulk load to firebase

In [39]:
#firebase
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
#service as account
credential_path='/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality/firebase.json'
credential = credentials.Certificate(credential_path)
firebase_admin.initialize_app(credential)
db = firestore.client()

In [161]:
#sec_result reload
source_path = '/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality/sec_result.xlsx'
source = pd.read_excel(source_path)
source.sample(8)


,email,rut,address,phone,licence,surnames,names,gender,movil,city,region,birth,service
11391,felipe.moreno.c@hotmail.com,17520271-4,"Avenida Los Carrera 2101, K41, Condominio Bell...",NaN,3,Moreno Contreras,Felipe Andrés,male,5.695076e+10,Quilpué,Valparaíso,1990,gas
9518,NaN,16401251-4,"Serrano 530, Población Estación , Nogales",332263431,B,Sánchez Pacheco,Juan Eduardo,male,5.696505e+10,Nogales,Valparaíso,1986,eli
11124,claudiocruzg@gmail.com,11676621-3,"Echaurren 437, 41, Block 1, Santiago",226895941,1,Cruz Garrido,Claudio Andrés,male,5.697913e+10,Santiago,Metropolitana de Santiago,1971,gas
1007,yo_isa_4@hotmail.com,11732758-2,"Psje El Chañar 626, Pobl Marga Marga, Ex-villa...",NaN,D,Hernández Brito,Bárbara Isabel,female,5.694216e+10,Villa Alemana,Valparaíso,1971,eli
5670,ivanromeroreyes@gmail.com,15068548-6,"Antupiren 9098, 32, Penalolén",0229378772,D,Romero Reyes,Ivan Roberto,male,5.697529e+10,Peñalolén,Metropolitana de Santiago,1982,eli
10091,h_tito_toro@hotmail.com,7244282-2,"Las Carpas 2679, Poblacion Mauque, Iquique",NaN,3,Toro Ossandón,Héctor Alejandro,male,5.699328e+10,Iquique,Tarapacá,1956,gas
5190,jps@enerted.cl,16959958-0,"Loma Del Durazno 1035, Pirque",979051548,A,Schroh Cruces,Juan Pablo,male,5.697905e+10,Pirque,Metropolitana de Santiago,1988,eli
4442,dgfuentes85@gmail.com,15748672-1,"Embalse El Yeso 6012, Penalolén",NaN,A,Fuentes Manríquez,Diego Ignacio,male,5.693087e+10,Peñalolén,Metropolitana de Santiago,1984,eli


In [162]:
source = source.replace({np.nan:None})

In [163]:
#database references
providers_collection = db.collection('energy_providers').document('source_providers').collection('providers')

In [164]:
from pytz import timezone
from traitlets.traitlets import Dict
import datetime
#document converter
def provider_converter(it):
  timestamp=datetime.datetime.now(tz=timezone('America/Santiago'))

  return {
      "rut":it.rut,
      "personal":{"name":it.names,"surname":it.surnames,"birth":it.birth,"gender":it.gender},
      "licence":{"category":it.licence,"service":it.service},
      "address":{"street":it.address,"city":it.city,"region":it.region},
      "contact":{
          "phone": int(it.phone) if it.phone!=None else None ,
          "movil":int(it.movil) if it.movil!=None else None ,
          "email":it.email
          },
      "update_at":timestamp,
  }



Bulk upload, test 2 times, and test again, and after go in peace

In [165]:
from google.cloud.firestore_v1.base_query import FieldFilter
#load
sized = source.head(20)
for tech in sized.itertuples():
  #fetch if exists
  docs = providers_collection.where(filter=FieldFilter('rut','==',tech.rut)).stream()
  docs_list = list(docs)
  #if exist 1 update
  if len(docs_list) == 1:
    put_doc = docs_list[0]
    providers_collection.document(put_doc.id).set(provider_converter(tech),merge=True)
    print('updated',put_doc.id)
  #if not with 0 create
  if len(docs_list) == 0:
    post_doc = providers_collection.document()
    post_doc.set(provider_converter(tech),merge=True)
    print('created',post_doc.id)






updated bXCsE9nTkIV8M60qFWyJ
updated ZPzhDs4tKfVDUqT5utki
updated VrzwqYckzQmUREfpdOMT
updated A35SHEYU3yGyMP30ftZn
updated 5Nd6RNgRn177ys8bJhxA
updated slfQpUOwt95l9XC2U2Eg
updated a1FCzYxjNp9Z3xtj927Z
updated QJIKWB0WwUhP7nEVV5z7
updated zMerTy8Hpgk9RUR4YP94
updated 95PXTqLiq1qd6vk42AZ4
updated DXRADsVfxAc4LICDFsWc
updated WjP4I0chikqUULktI4t4
updated Ec7XzZeeFDDnhSj6nu18
updated CbceKRwBLXsgVF3ekmOb
updated dp6dlMcyjnuIqJaeDzdH


ValueError: ignored